MongoDB with Python

In [66]:
from pymongo import MongoClient, errors

In [9]:
# init client to connect to mongoDB
my_client = MongoClient('mongodb://localhost:27017')

In [16]:
my_client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [24]:
# connect to db
my_client.list_database_names()

['SoundCloud', 'admin', 'auth_tutorial', 'config', 'local', 'store']

In [31]:
# connect to db store
db_store = my_client.store

In [26]:
db_store

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'store')

In [40]:
# list collections from store db
print(db_store.list_collection_names())

['admin', 'users']


In [41]:
users_col = db_store.users

In [42]:
users_col

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'store'), 'users')

C - Create

In [36]:
users_col.insert_one({
    "name": "John",
    "age": 29
})

In [38]:
new_user = {
    "name": "Keanu Reeves",
    "age": 57
}

In [39]:
users_col.insert_one(new_user)

In [43]:
users_col

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'store'), 'users')

R - Read

In [46]:
users_col.find_one({"name":"Keanu Reeves"})

{'_id': ObjectId('668a85a873c8b630e2131b6a'),
 'name': 'Keanu Reeves',
 'age': 57}

In [47]:
users_col.find_one(new_user)

{'_id': ObjectId('668a85a873c8b630e2131b6a'),
 'name': 'Keanu Reeves',
 'age': 57}

U- Update

In [48]:
users_col.update_one({"name":"John"}, {"$set":{"name":"Will", "age":53}})

D - Delete

In [49]:
users_col.delete_one({"name":"Will"})

In [53]:
products_db = db_store.products

In [51]:
bag = {"name": "Bag", "price":23.78}

In [52]:
products_db.insert(bag)

/var/folders/1l/xv3wqjt14cgf1mmyz0bm3bnr0000gn/T/ipykernel_12209/1657952950.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  products_db.insert(bag)


ObjectId('668a889073c8b630e2131b6b')

In [55]:
phone = {"name":"iphone 11", "price": 800}
notebook = {"name":"Notebook", "price": 4.99}

In [56]:
products_db.insert_many([phone, notebook])

In [58]:
pen = {"name":"Pen2", "price":1.22}
insert_pen = products_db.insert_one(pen)

In [59]:
print(insert_pen.acknowledged)
print(insert_pen.inserted_id)

True
668a8a2173c8b630e2131b6f


In [60]:
water_bottle = {"name":"Water Bottle", "price": 14.5}
jar = {"name":"Jar", "price":4.99}
insert_several = products_db.insert_many([water_bottle, jar])
insert_several

In [61]:
print(insert_several.acknowledged)
print(insert_several.inserted_ids)


True
[ObjectId('668a8ac273c8b630e2131b70'), ObjectId('668a8ac273c8b630e2131b71')]


Error Handling When Inserting

In [67]:
try:
    products_db.insert_one(pen)
except errors.DuplicateKeyError:
    print("Duplicate Entry Detected")

Duplicate Entry Detected


In [69]:
try:
    products_db.insert_many([water_bottle, jar])
except errors.BulkWriteError:
    print("Bulk Error Detected")

Bulk Error Detected


In [70]:
mug = {"name":"Mug", "price":2.33}
try:
    products_db.insert_many([mug, jar])
except errors.BulkWriteError:
    print("Bulk Error Detected")

Bulk Error Detected


In [71]:
products_db

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'store'), 'products')